In [2]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(),"..")))

from Datos.Data_acq import get_mnist_data
#import Data_acq as data
from models import Encoder
from models import Decoder
from models import cVAE
import matplotlib.pyplot as plt
import importlib

import importlib
import numpy as np
importlib.reload(Encoder)
importlib.reload(Decoder)
import tensorflow as tf
from keras.callbacks import EarlyStopping
importlib.reload(cVAE)
import importlib
#importlib.reload(data)
dataset="fashion"
x_train, x_test,x_val ,y_train, y_test,y_val, x_train_1, y_train_1 = get_mnist_data(dataset=dataset)

original_dim = 28*28
beta = 1.0

train_dataset = tf.data.Dataset.from_tensor_slices(((x_train, y_train), x_train))
train_dataset = train_dataset.batch(128)

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=3,          
    restore_best_weights=True  
)

val_dataset = tf.data.Dataset.from_tensor_slices(((x_val, y_val), x_val))
val_dataset = val_dataset.batch(128)

#intermediate_dim=[128,256,512,1024]
#latent_dim=[2, 64, 128, 256]
intermediate_dim=[256]
latent_dim=[128]
  
for i in range(len(intermediate_dim)):
    en = Encoder.build_enconder(intermediate_dim=intermediate_dim[i],latent_dim=latent_dim[i])
    de = Decoder.build_decoder(intermediate_dim=intermediate_dim[i],latent_dim=latent_dim[i])
        
    cvae = cVAE.CVAE(encoder=en, decoder=de, original_dim=original_dim, beta=beta)
    cvae.compile(optimizer=tf.keras.optimizers.Adam())

    cvae.fit(train_dataset,
            epochs=10,
            batch_size=128,
            validation_data=val_dataset,
            callbacks=[early_stopping]
            )

    en.save(f"en_int_{intermediate_dim[i]}_lat_{latent_dim[i]}_{dataset}.keras")
    de.save(f"de_int_{intermediate_dim[i]}_lat_{latent_dim[i]}_{dataset}.keras")




Usando fashion como dataset
Epoch 1/10
430/430 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - kl_loss: 4.2654 - loss: 61.6062 - reconstruction_loss: 57.3408 - val_kl_loss: 6.1082 - val_loss: 35.5090 - val_reconstruction_loss: 29.3139
Epoch 2/10
430/430 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - kl_loss: 5.7885 - loss: 33.8934 - reconstruction_loss: 28.1049 - val_kl_loss: 5.1594 - val_loss: 30.7379 - val_reconstruction_loss: 25.6539
Epoch 3/10
430/430 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - kl_loss: 4.8640 - loss: 29.8375 - reconstruction_loss: 24.9735 - val_kl_loss: 4.6644 - val_loss: 28.2349 - val_reconstruction_loss: 23.6483
Epoch 4/10
430/430 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - kl_loss: 4.5587 - loss: 27.7192 - reconstruction_loss: 23.1605 - val_kl_loss: 4.5341 - val_loss: 26.8835 - val_reconstruction_loss: 22.3468
Epoch 5/10
430/430 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - kl_loss: 4.6213 - loss: 26.5412 - reconstruction_loss: 21.9199 - val_kl_loss: 4.7330 - val_loss: 25.6217 - val_reconstruction_loss: 20.9489
Ep

In [ ]:
from keras.models import load_model
from custom_layers.Sampling import Sampling  # Si usás esta capa personalizada

en1 = load_model(f"encoders/en_int_512_lat_256_mnist.keras", custom_objects={"Sampling": Sampling})
de1 = load_model(f"decoders/de_int_512_lat_256_mnist.keras")
cvae1 = cVAE.CVAE(encoder=en1, decoder=de1, original_dim=original_dim, beta=1)
cvae1.compile(optimizer=tf.keras.optimizers.Adam())

In [16]:
en.save(f"en_int_{intermediate_dim[i]}_lat_{latent_dim[i]}_{dataset}.keras")
de.save(f"de_int_{intermediate_dim[i]}_lat_{latent_dim[i]}_{dataset}.keras")



In [7]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),"..")))


from Datos.Data_acq import get_mnist_data
#import Data_acq as data
from models import Encoder
from models import Decoder
from models import cVAE
import matplotlib.pyplot as plt
import importlib
import tensorflow as tf

from keras.callbacks import EarlyStopping
import models.Predictor as predictores

dataset="fashion"
x_train, x_test,x_val ,y_train, y_test,y_val, x_train_1, y_train_1 = get_mnist_data(dataset=dataset)


predictor = predictores.build_predictor() 


optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

predictor.compile(optimizer, loss="categorical_crossentropy")

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=3,          
    restore_best_weights=True  
)

#x_rain_conv = np.reshape(x_train,(28,28,1)) 
predictor.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),  # Include validation data
    callbacks=[early_stopping]  # Add early_stopping to callbacks
)



Usando fashion como dataset


Model: "predictor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ original_input (InputLayer)     │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_layer_16 (ReshapeLayer) │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 14, 14, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 14, 14, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 7, 7, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 7, 7, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │        31,370 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,570 (197.54 KB)

 Trainable params: 50,378 (196.79 KB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - loss: 0.8191 - val_loss: 0.6170
Epoch 2/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - loss: 0.4207 - val_loss: 0.3553
Epoch 3/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 12s 27ms/step - loss: 0.3586 - val_loss: 0.3278
Epoch 4/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - loss: 0.3190 - val_loss: 0.2700
Epoch 5/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 23s 54ms/step - loss: 0.2998 - val_loss: 0.2756
Epoch 6/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - loss: 0.2892 - val_loss: 0.2712
Epoch 7/100
430/430 ━━━━━━━━━━━━━━━━━━━━ 16s 37ms/step - loss: 0.2692 - val_loss: 0.2769
